In [1]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import pandas as pd
import csv
import time

# 設定方法
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

# 人臉偵測設定
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='model/face_landmarker.task'),
    running_mode=VisionRunningMode.IMAGE)

landmarks_data = []

# 定義連線集合
face_mesh_connections = mp.solutions.face_mesh.FACEMESH_TESSELATION

start_time = time.time()

# 執行人臉偵測
with FaceLandmarker.create_from_options(options) as landmarker:
    cap = cv2.VideoCapture(0)  # 讀取攝影鏡頭
    while True:
        ret, frame = cap.read()  # 讀取影片的每一幀
        if not ret:
            print("Cannot receive frame")
            break

        w, h = frame.shape[1], frame.shape[0]
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        face_landmarker_result = landmarker.detect(mp_image)

        face_landmarks_list = face_landmarker_result.face_landmarks
        annotated_image = np.copy(frame)

        # 獲取每個臉的 landmarks 座標及連線資訊
        for face_landmarks in face_landmarks_list:
            landmarks_df = pd.DataFrame(columns=["x", "y", "z"])
            for idx, landmark in enumerate(face_landmarks):
                
                landmarks_df = landmarks_df.append({
                    "x": landmark.x * w,
                    "y": landmark.y * h,
                    "z": landmark.z * w
                }, ignore_index=True)

            # 繪製 landmarks 和連線
            face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
            face_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
            ])
            solutions.drawing_utils.draw_landmarks(
                image=annotated_image,
                landmark_list=face_landmarks_proto,
                connections=face_mesh_connections,
                landmark_drawing_spec=None,
                connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style()
            )

        cv2.imshow('oxxostudio', annotated_image)


        # 每次迭代檢查是否超過 5 秒
        if time.time() - start_time > 5:
            break

        if cv2.waitKey(10) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    
# 儲存 landmarks 和連線資訊到 CSV
with open('face_landmarks.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for landmarks in landmarks_data:
        csvwriter.writerow([f"{x},{y},{z}" for x, y, z in landmarks])

2024-10-28 00:52:59.564306: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0000 00:00:1730047984.576078       1 face_landmarker_graph.cc:169] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-10-28 00:53:04.863 Python[5259:1742187] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-10-28 00:53:06.879 Python[5259:1742187] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000

In [2]:
landmarks_df

,x,y,z
0,687.257996,380.275784,-30.396616
1,686.550980,343.312433,-52.823620
2,686.814804,355.226977,-28.418615
3,679.118729,312.901847,-37.761080
4,686.604614,333.472395,-55.838246
...,...,...,...
473,745.190735,285.493340,4.374522
474,754.400177,285.503469,4.379108
475,745.304337,277.051034,4.379698
476,735.634232,285.274193,4.369528


In [3]:
landmarks_df.to_csv("face_landmarks.csv", index=False)

In [8]:
data = pd.read_csv('face_connections.csv')
data[:0]

,"18,17","82,38","8,9","456,248","167,2","303,271","69,104","253,450","41,42","315,16",...,"334,282","201,18","33,25","166,60","251,284","226,111","169,214","325,318","450,349","40,185"


In [ ]:
data[:0].to_csv('face_connections.csv', index=True) 